<a href="https://colab.research.google.com/github/znumunz/TMAO/blob/main/TMAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**[github](https://github.com/znumunz/TMAO)**

#**Import Library**

In [ ]:
from google.colab import drive
import pandas as pd
pd.set_option('display.max_columns', None)   # แสดงทุกคอลัมน์
# pd.set_option('display.width', None)         # ให้ความกว้างไม่จำกัด
# pd.set_option('display.max_colwidth', None)  # ถ้าค่าในคอลัมน์ยาว ก็แสดงทั้งหมด
import string #ใช้สำหรับการเขียนฟังก์ชัน เลือกคอลัมน์ด้วย ชื่อคอลัมน์ A B C


#**Read Data**

In [ ]:
drive.mount('/content/drive')

In [ ]:
# สร้างทางลัดของ path ไปยัง "My Drive" ก่อน จึงจะสามารถอ่านไฟล์จาก path เดียวกันได้

path = "/content/drive/MyDrive/TMAO/Data/__อันนี้_ล่าสุด_Nov_4_2025__of_EGAT04102024_hs_trop_I_AJVICHAI_11.xlsx"
df = pd.read_excel(path)
df.head(3)

#**Choose columns**

In [ ]:
# เลือกคอลัมน์จากชื่อคอลัมน์
data = df[['empn','case','age','sex']]
data

## ฟังก์ชันการเลือกคอลัมน์แบบใช้ชื่อ A B C:F **ได้ในครั้งเดียว**

In [ ]:
def excel_idx(s):
    n = 0
    for c in s.upper():
        n = n * 26 + (ord(c) - 64)
    return n - 1

def pick(df, *args):
    idx = []
    for a in args:
        if ":" in a:
            s, e = a.split(":")
            idx.extend(range(excel_idx(s), excel_idx(e)+1))
        else:
            idx.append(excel_idx(a))
    return df.iloc[:, idx]


In [ ]:
df = pick(df,"V","Y","W","AC","AO","C:S")
df

#**Clean Data**

In [ ]:
df.info()

In [ ]:
df['dead all cause อันนี้'].value_counts()

#**Feature Engineering**

## **Feature Extraction แยกวัน เดือน ปี**

In [ ]:
df

##**คำนวณช่วงเวลาที่มีชีวิต คนที่มีชีวิต สิ้นสุดการรักษา 30/12/2019**

##**แยกกลุ่มเสียชีวิต กับไม่เสียชีวิต**

In [ ]:
df1 = df[df['dead all cause อันนี้']==1]
df1.head(3)

In [ ]:
df0 = df[df['dead all cause อันนี้']==0]
df0.head(3)

#**Data Exploration**